In [1]:
from openai import OpenAI

In [2]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
import os

In [6]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [7]:
from datetime import datetime

In [8]:
import requests

In [9]:
from supabase import create_client, Client

In [10]:
supabase_url = os.environ.get("SUPABASE_URL")

In [11]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [12]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [90]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls, news_date')\
.gte('news_date', '2026-02-10')\
.lte('news_date', '2026-02-11')\
    .order('created_at', desc=False)\
    .execute()

result.data

[{'id': 326,
  'news_output': 'Dream Companion announced a new advanced conversational AI companion platform featuring long-term memory and personalized interactions (press release / GlobeNewswire coverage).',
  'source_urls': ['https://www.manilatimes.net/2026/02/10/tmt-newswire/globenewswire/dream-companion-launches-advanced-ai-companion-platform-featuring-long-term-memory-and-personalized-interaction/2274695'],
  'news_date': '2026-02-10'},
 {'id': 327,
  'news_output': 'Botgauge AI, a US-based autonomous software-testing startup, raised $2 million led by Surface Ventures (participation from IA Seed Ventures and Saka Ventures); funds to expand R&D and scale operations (Economic Times coverage).',
  'source_urls': ['https://m.economictimes.com/tech/funding/software-testing-firm-botgauge-ai-raises-2-million-led-by-surface-ventures/articleshow/128122718.cms'],
  'news_date': '2026-02-10'},
 {'id': 328,
  'news_output': 'Several Latin American institutions launched Latam-GPT, an open-so

In [112]:
research_data = [
 {'id': 343,
  'news_output': 'Tem (AI-native energy platform) closed a $75 million Series B led by Lightspeed, valuing the company at >$300M and funding expansion into the US and Australia; published Feb 10, 2026.',
  'source_urls': ['https://thenextweb.com/news/tem-raises-75-m-to-automate-energy-markets-with-ai-first-platform'],
  'news_date': '2026-02-10'},
 {'id': 344,
  'news_output': 'Splunk announced Splunk-hosted AI models (hosted generative models integrated into the Splunk Platform) with General Availability targeted/announced for February 18, 2026 — a product milestone adding built-in AI capability for observability and security.',
  'source_urls': ['https://www.splunk.com/en_us/blog/platform/splunk-platform-building-the-data-foundation-for-agentic-ai.html'],
  'news_date': '2026-02-18'},
 {'id': 345,
  'news_output': 'Alphabet has sold $20 billion in a multi-part bond offering to help fund expanded AI spending — including data centers and AI chips — signaling a major capital-raising move to support its AI infrastructure investments.',
  'source_urls': ['https://www.reuters.com/business/alphabet-sells-bonds-worth-20-billion-fund-ai-spending-2026-02-10/'],
  'news_date': '2026-02-10'},
 {'id': 346,
  'news_output': "China's Alibaba announced a new AI model called RynnBrain focused on powering robots and 'physical AI', signaling a push into robotics-specific models and real-world automation.",
  'source_urls': ['https://www.cnbc.com/2026/02/10/alibaba-ai-model-robotics-rynnbrain-china.html'],
  'news_date': '2026-02-10'},
 {'id': 347,
  'news_output': "Cisco announced expanded 'AgenticOps' innovations across networking, security, and observability, with Agentic capabilities starting to roll out in February 2026.",
  'source_urls': ['https://newsroom.cisco.com/content/r/newsroom/en/us/a/y2026/m02/cisco-expands-agenticops-innovations-across-portfolio.html'],
  'news_date': '2026-02-10'},
 {'id': 348,
  'news_output': 'Bretton AI (formerly Greenlite) announced a $75 million Series B funding round and rebrand, expanding its AI offerings (company press/PR coverage).',
  'source_urls': ['https://finance.yahoo.com/news/bretton-ai-raises-75m-series-133000430.html'],
  'news_date': '2026-02-10'},
 {'id': 349,
  'news_output': 'Thomson Reuters announced the acquisition of Noetica, an AI-native platform for corporate transaction intelligence, via company press release.',
  'source_urls': ['https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence'],
  'news_date': '2026-02-10'},
 {'id': 350,
  'news_output': 'Runway raised $315 million at a $5.3 billion valuation to expand its AI video capabilities and pursue more capable world models (TechCrunch coverage).',
  'source_urls': ['https://techcrunch.com/2026/02/10/ai-video-startup-runway-raises-315m-at-5-3b-valuation-eyes-more-capable-world-models/'],
  'news_date': '2026-02-10'},
 {'id': 351,
  'news_output': 'Qureos secured a $5 million seed round to expand its AI-driven hiring platform across global markets (industry press coverage).',
  'source_urls': ['https://theaiinsider.tech/2026/02/10/qureos-secures-5m-seed-round-to-scale-ai-hiring-platform-across-global-markets/'],
  'news_date': '2026-02-10'},
 {'id': 352,
  'news_output': 'Gather AI raised $40 million in funding to scale its physical AI platform for global logistics; round led by Smith Point Capital Management, bringing total funding to roughly $74M.',
  'source_urls': ['https://www.freightwaves.com/news/gather-ai-raises-40m-to-scale-its-physical-ai-platform-for-global-logistics'],
  'news_date': '2026-02-10'},
 {'id': 353,
  'news_output': 'Simple AI announced a $14 million funding round (from First Harmonic and Y Combinator) to build voice AI agents for sales and related automated voice agent capabilities.',
  'source_urls': ['https://venturebeat.com/business/simple-ai-announces-14m-from-first-harmonic-and-y-combinator-to-build-voice-ai-agents-that-sell'],
  'news_date': '2026-02-10'},
 {'id': 354,
  'news_output': "IBM introduced a next-generation FlashSystem portfolio described as 'Autonomous Storage' powered by agentic AI capabilities — a notable product/feature announcement for enterprise AI infrastructure.",
  'source_urls': ['https://newsroom.ibm.com/next-generation-ibm-flashsystem-portfolio'],
  'news_date': '2026-02-10'},
 {'id': 355,
  'news_output': 'Vibrant Capital launched to scale AI in real-economy workflows, a new fund/initiative aimed at expanding AI adoption in industry verticals.',
  'source_urls': ['https://finance.yahoo.com/news/vibrant-capital-launches-scale-ai-150200808.html'],
  'news_date': '2026-02-10'},
 {'id': 356,
  'news_output': 'Reuters (Feb 10, 2026) reports that AI disruption fears created a buying opportunity in US software stocks and specifically notes that wealth-management startup Altruist introduced AI-enabled tax planning features — a new feature announcement for AI in financial services.',
  'source_urls': ['https://www.reuters.com/business/ai-disruption-fears-create-buying-chance-us-software-stocks-strategists-say-2026-02-10/'],
  'news_date': '2026-02-10'},
 {'id': 357,
  'news_output': 'Israel-based FINQ launched two U.S. ETFs that will be managed solely by AI — an industry-first product announcement applying AI to ETF management.',
  'source_urls': ['https://www.reuters.com/business/israels-finq-launches-us-etfs-managed-solely-by-ai-2026-02-10/'],
  'news_date': '2026-02-10'},
 {'id': 358,
  'news_output': 'Bretton AI (formerly Greenlite AI) raised $75 million in a Series B to expand deployment of AI agents for financial crime detection — notable funding and application-specific deployment.',
  'source_urls': ['https://www.pymnts.com/news/investment-tracker/2026/bretton-ai-secures-75-million-to-deploy-ai-agents-against-financial-crime/'],
  'news_date': '2026-02-10'},
 {'id': 359,
  'news_output': 'Israel-based FINQ launched two U.S. ETFs that will be managed solely by AI — an industry-first product announcement applying AI to ETF management.',
  'source_urls': ['https://www.reuters.com/business/israels-finq-launches-us-etfs-managed-solely-by-ai-2026-02-10/'],
  'news_date': '2026-02-10'},
 {'id': 360,
  'news_output': 'Bretton AI (formerly Greenlite AI) raised $75 million in a Series B to expand deployment of AI agents for financial crime detection — notable funding and application-specific deployment.',
  'source_urls': ['https://www.pymnts.com/news/investment-tracker/2026/bretton-ai-secures-75-million-to-deploy-ai-agents-against-financial-crime/'],
  'news_date': '2026-02-10'},
 {'id': 361,
  'news_output': 'Bloomberg reports Multiverse Computing is in talks to raise about €500 million in fresh funding that would value the Spanish AI company at over €1.5 billion; financing expected to close in H1 2026 (people familiar with the matter).',
  'source_urls': ['https://www.bloomberg.com/news/articles/2026-02-10/ai-firm-multiverse-said-to-hit-1-5-billion-value-with-new-funds'],
  'news_date': '2026-02-10'},
 {'id': 362,
  'news_output': 'Bloomberg reports Multiverse Computing is in talks to raise about €500 million in fresh funding that would value the Spanish AI company at over €1.5 billion; financing expected to close in H1 2026 (people familiar with the matter).',
  'source_urls': ['https://www.bloomberg.com/news/articles/2026-02-10/ai-firm-multiverse-said-to-hit-1-5-billion-value-with-new-funds'],
  'news_date': '2026-02-10'},
 {'id': 363,
  'news_output': "Chile launched 'Latam-GPT', an open-source language model trained with Latin American cultural data — positioned as a region-focused LLM for Spanish/Portuguese contexts.",
  'source_urls': ['https://www.ctpost.com/news/world/article/chile-launches-open-source-ai-model-designed-for-21345504.php'],
  'news_date': '2026-02-10'},
 {'id': 364,
  'news_output': 'NIST announced awards totaling over $3 million to small businesses advancing technologies including AI, biotechnology, semiconductors and quantum — includes AI-focused grants supporting commercialization and R&D.',
  'source_urls': ['https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology'],
  'news_date': '2026-02-10'},
 {'id': 365,
  'news_output': 'Berget AI announced a $2.4 million funding round to build a sovereign AI infrastructure platform in Europe, a funding event tied to sovereign/cloud AI infrastructure efforts.',
  'source_urls': ['https://theaiinsider.tech/2026/02/10/berget-ai-announces-2-4m-funding-round-to-build-sovereign-ai-infrastructure-platform-in-europe/'],
  'news_date': '2026-02-10'},
 {'id': 366,
  'news_output': "Anytime AI launched Anytime AI 2.0 with new features (example: 'Talk to Teddy') — a product update/new feature release for an AI legal-tech product focused on plaintiff attorneys.",
  'source_urls': ['https://finance.yahoo.com/news/anytime-ai-launches-2-0-124700364.html'],
  'news_date': '2026-02-10'}]

In [81]:
research_input = [{'id': 328,
  'news_output': 'Several Latin American institutions launched Latam-GPT, an open-source regional large language model based on Llama 3 tech aimed at supporting local languages and public-service use cases (Firstpost coverage of Feb 10 launch).',
  'source_urls': ['https://www.firstpost.com/explainers/firstup-latin-america-to-launch-its-own-ai-model-and-other-news-today-13977896.html'],
  'news_date': '2026-02-10'},
 {'id': 331,
  'news_output': 'Algolia announced participation at AI Day France 2026 (Businesswire/press coverage) — representative of events/industry attention but not a product/model launch by Algolia itself.',
  'source_urls': ['https://www.businesswire.com/news/home/20260210094401/en/Beyond-Model-Benchmarks-Algolia-CTO-Xavier-Grand-Joins-AI-Day-France-2026-Panel-on-Algolias-Role-in-the-GenAI-UX-Revolution'],
  'news_date': '2026-02-10'},
 {'id': 332,
  'news_output': 'UK chipmaker Fractile announced a plan to invest about $136 million to boost AI chip production capacity — a government-backed/industry-backed scaling move to increase domestic AI chip output.',
  'source_urls': ['https://www.reuters.com/world/uk/uk-chipmaker-fractile-plans-136-million-boost-ai-chip-production-2026-02-10/'],
  'news_date': '2026-02-10'},
 {'id': 333,
  'news_output': 'The photorealistic AI feature film "Deep Frame" premiered/followed an industry screening on Feb 10, 2026 — marketed as the world’s first photorealistic AI-rendered feature film, notable as an AI-driven creative milestone.',
  'source_urls': ['https://deepframemovie.com/'],
  'news_date': '2026-02-10'},
 {'id': 334,
  'news_output': 'WSJ opinion coverage reported Elon Musk’s stated ambition to launch a large number of solar-powered AI data centers into orbit — a high-profile, strategic plan/ambition that could signal future capital and infrastructure moves in AI deployment.',
  'source_urls': ['https://www.wsj.com/opinion/elon-musk-xai-spacex-solar-powered-data-centers-f78fe95d?gaa_at=eafs'],
  'news_date': '2026-02-10'},
 {'id': 336,
  'news_output': 'International Federation of Robotics (IFR) published a new position paper on AI in robotics (Feb 10, 2026), signalling industry guidance/advocacy as robots powered by AI move further into commercial use — notable for policy/industry feature developments.',
  'source_urls': ['https://ifr.org/ifr-press-releases/news/ai-in-robotics-new-position-paper'],
  'news_date': '2026-02-10'},
 {'id': 337,
  'news_output': 'Oracle announced dozens of role-based AI agents embedded across its Fusion Cloud Applications and unveiled AI-centric NetSuite Next (role-based agents, agentic workflows and analytics) — a significant feature expansion for enterprise AI in ERP/CRM. Source: Oracle PR (PR Newswire) and contemporaneous coverage (SiliconANGLE, ITBrief).',
  'source_urls': ['https://www.prnewswire.com/news-releases/oracle-ai-agents-help-marketing-sales-and-service-leaders-enhance-customer-experiences-302681980.html'],
  'news_date': '2026-02-10'},
 {'id': 338,
  'news_output': 'AlphaLum raised CHF 3.4 million to scale its optics and sensing technologies for smart glasses — a funding event tied to AI-enabled sensing and hardware for AR/VR applications.',
  'source_urls': ['https://ggba.swiss/en/alphalum-raises-chf-3-4-million-to-scale-optics-and-sensing-technologies-for-smart-glasses/'],
  'news_date': '2026-02-10'},
 {'id': 339,
  'news_output': "Reuters reports SoftBank — which invested >$30B in OpenAI in 2025 — is in talks to invest an additional up to $30 billion in OpenAI; coverage frames this as a potential major infusion that would materially affect OpenAI's funding/backing and SoftBank's earnings outlook.",
  'source_urls': ['https://www.reuters.com/world/asia-pacific/softbank-earnings-set-openai-boost-with-focus-future-funding-2026-02-10/'],
  'news_date': '2026-02-10'},
 {'id': 340,
  'news_output': 'Cisco announced multiple AI-era infrastructure and security innovations including a new AI networking chip/router to accelerate data-center AI traffic and a suite of security capabilities aimed at safely adopting agentic AI in enterprise environments.',
  'source_urls': ['https://newsroom.cisco.com/content/r/newsroom/en/us/a/y2026/m02/cisco-launches-breakthrough-innovations-for-the-ai-era.html'],
  'news_date': '2026-02-10'}]
research_output = []

In [94]:
def openai_research_v2(research_input):
    for i in research_input:
        response = client.responses.create(
            model = "gpt-5-nano",
            tools = [{
                "type": "web_search"
            }],
            include = ["web_search_call.action.sources"],
            input = f"""You are an AI research analyst for Krux.
            Your job is to create a fact-checked research brief for a given AI news event.
            
            APPROACH:
            1. Start with WHAT IS NEW — the specific event, announcement, or development.
            2. Add ONLY the context needed to understand why this matters.
            3. For funding/M&A: focus on the new round, amount, investors, and valuation. 
               Do NOT include historical funding rounds or prior valuations.
            4. For product/launch news: include key technical details, who built it, 
               and what problem it solves.
            5. Maximum 8-10 bullet points. If you need more, you're being redundant.
            
            CRITICAL RULES:
            - Every fact must be traceable to a credible published source.
            - No point should repeat information from another point.
            - No opinions, predictions, or analysis.
            - No fluff — get straight to the facts.
            - First bullet must always state the core new development.
            
            OUTPUT: Bullet points with inline source citations. Nothing else.

            Here is what you need to research on:

            {i['news_output']}
            """,
        )

        output = response.output_text
        print(output)

        final_dictionary = {
            'event_id': f"{i['id']}_{i['news_date']}",
            'news_date': i['news_date'],
            'output': output,
            'model_provider': 'openai'
        }

        research_output.append(final_dictionary)
        print(final_dictionary)

        # save_research(final_dictionary)

    return research_output

In [113]:
openai_research_v2(research_data)

- Tem closes a $75 million oversubscribed Series B led by Lightspeed Venture Partners to accelerate its US and Australia expansion, with a valuation reported at over $300 million. ([techcrunch.com](https://techcrunch.com/2026/02/09/tem-raises-75m-to-remake-electricity-markets-using-ai/))

- The round includes participation from AlbionVC, Allianz, Atomico, Hitachi Ventures, Schroders Capital, Voyager Ventures, and Revent; Lightspeed partner Paul Murphy to join Tem’s board. ([techcrunch.com](https://techcrunch.com/2026/02/09/tem-raises-75m-to-remake-electricity-markets-using-ai/))

- Tem plans to use the capital to scale its AI-native energy platform globally, starting with entry into the United States (initial focus on Texas) and Australia. ([thenextweb.com](https://thenextweb.com/news/tem-raises-75-m-to-automate-energy-markets-with-ai-first-platform))

- Tem’s core offering is an AI-native energy platform that automates pricing, matching, and execution of electricity transactions, buil

[{'event_id': '328_2026-02-10',
  'news_date': '2026-02-10',
  'output': '- Latam-GPT was officially launched on February 10, 2026, in Chile as the region’s open-source large language model. The launch is described as Latam-GPT being “el primer Gran Modelo de Lenguaje Abierto de América Latina y el Caribe.” ([caf.com](https://www.caf.com/es/actualidad/eventos/lanzamiento-latam-gpt-primer-gran-modelo-de-lenguaje-abierto-de-america-latina-y-el-caribe/))\n\n- The project is a Latin American collaboration coordinated by Chile’s National Center for Artificial Intelligence (CENIA) with input from more than 60 institutions across the region (and around 200 specialists). CAF’s event page states the effort is built “de manera colaborativa por más de 60 instituciones y cerca de 200 especialistas de la región.” ([caf.com](https://www.caf.com/es/actualidad/eventos/lanzamiento-latam-gpt-primer-gran-modelo-de-lenguaje-abierto-de-america-latina-y-el-caribe/))\n\n- Latam-GPT is described as an open-so

In [114]:


for i in research_output:
    
print(research_output)

[{'event_id': '328_2026-02-10', 'news_date': '2026-02-10', 'output': '- Latam-GPT was officially launched on February 10, 2026, in Chile as the region’s open-source large language model. The launch is described as Latam-GPT being “el primer Gran Modelo de Lenguaje Abierto de América Latina y el Caribe.” ([caf.com](https://www.caf.com/es/actualidad/eventos/lanzamiento-latam-gpt-primer-gran-modelo-de-lenguaje-abierto-de-america-latina-y-el-caribe/))\n\n- The project is a Latin American collaboration coordinated by Chile’s National Center for Artificial Intelligence (CENIA) with input from more than 60 institutions across the region (and around 200 specialists). CAF’s event page states the effort is built “de manera colaborativa por más de 60 instituciones y cerca de 200 especialistas de la región.” ([caf.com](https://www.caf.com/es/actualidad/eventos/lanzamiento-latam-gpt-primer-gran-modelo-de-lenguaje-abierto-de-america-latina-y-el-caribe/))\n\n- Latam-GPT is described as an open-source

In [104]:
def save_research(research_json):
    supabase.table('research_assistant').insert({
        'event_id': research_json['event_id'],
        'model_provider': research_json['model_provider'],
        'news_date': research_json['news_date'],
        'output': research_json['output']
    }).execute()

In [116]:
for x in a:
    save_research(x)

In [115]:
a = [{'event_id': '343_2026-02-10', 'news_date': '2026-02-10', 'output': '- Tem closes a $75 million oversubscribed Series B led by Lightspeed Venture Partners to accelerate its US and Australia expansion, with a valuation reported at over $300 million. ([techcrunch.com](https://techcrunch.com/2026/02/09/tem-raises-75m-to-remake-electricity-markets-using-ai/))\n\n- The round includes participation from AlbionVC, Allianz, Atomico, Hitachi Ventures, Schroders Capital, Voyager Ventures, and Revent; Lightspeed partner Paul Murphy to join Tem’s board. ([techcrunch.com](https://techcrunch.com/2026/02/09/tem-raises-75m-to-remake-electricity-markets-using-ai/))\n\n- Tem plans to use the capital to scale its AI-native energy platform globally, starting with entry into the United States (initial focus on Texas) and Australia. ([thenextweb.com](https://thenextweb.com/news/tem-raises-75-m-to-automate-energy-markets-with-ai-first-platform))\n\n- Tem’s core offering is an AI-native energy platform that automates pricing, matching, and execution of electricity transactions, built around the Rosso transaction engine and the RED neo-utility interface. ([thenextweb.com](https://thenextweb.com/news/tem-raises-75-m-to-automate-energy-markets-with-ai-first-platform))\n\n- Tem asserts it can reduce energy costs for customers by up to 30% relative to traditional procurement, by removing multiple intermediaries from the transaction flow. ([techcrunch.com](https://techcrunch.com/2026/02/09/tem-raises-75m-to-remake-electricity-markets-using-ai/))\n\n- The company reports a UK customer base of over 2,600 businesses, including Boohoo Group, Fever-Tree, and Newcastle United FC. ([techcrunch.com](https://techcrunch.com/2026/02/09/tem-raises-75m-to-remake-electricity-markets-using-ai/))\n\n- In 2025, Tem claims to have processed approximately two terawatt-hours of electricity transactions, supporting its 2,600+ UK customers. ([thenextweb.com](https://thenextweb.com/news/tem-raises-75-m-to-automate-energy-markets-with-ai-first-platform))\n\n- Tem was founded in 2021 in London by Joe McDonald, Jason Stocks, Bartlomiej Szostek, and Ross Mackay, with the CEO quoted on the round’s strategic direction. ([eu-startups.com](https://www.eu-startups.com/2026/02/london-based-energy-transactions-scale-up-tem-raises-e62-9-million-to-become-the-stripe-of-energy/))\n\n- EU-Startups describes Tem’s ambition to become the “Stripe of energy,” underscoring its aim to provide open, AI-native energy transaction infrastructure at scale. ([eu-startups.com](https://www.eu-startups.com/2026/02/london-based-energy-transactions-scale-up-tem-raises-e62-9-million-to-become-the-stripe-of-energy/))', 'model_provider': 'openai'},
{'event_id': '344_2026-02-18', 'news_date': '2026-02-18', 'output': '- Splunk announced native hosted AI models with General Availability on February 18, 2026, integrated directly into the Splunk Cloud Platform. Models include Foundation-sec-1.1-8b-instruct (security-focused) and GPT OSS (20B and 120B). ([splunk.com](https://www.splunk.com/en_us/blog/platform/splunk-platform-building-the-data-foundation-for-agentic-ai.html?utm_source=openai))\n- Foundation-sec-1.1-8b-instruct is designed for security use cases, understanding logs, summarizing events, and suggesting detections in natural language. ([splunk.com](https://www.splunk.com/en_us/blog/platform/splunk-platform-building-the-data-foundation-for-agentic-ai.html?utm_source=openai))\n- GPT OSS variants (20B and 120B) are available for general data exploration and complex reasoning within the hosted model suite. ([splunk.com](https://www.splunk.com/en_us/blog/platform/splunk-platform-building-the-data-foundation-for-agentic-ai.html?utm_source=openai))\n- The Model Context Protocol (MCP) Server is GA as of February 4, 2026, providing a secure bridge between AI agents and Splunk data to enable rapid agent-data interactions. ([splunk.com](https://www.splunk.com/en_us/blog/platform/splunk-platform-building-the-data-foundation-for-agentic-ai.html?utm_source=openai))\n- Hosting these models in Splunk Cloud Platform means AI workloads run on GPU-backed, multi-tenant infrastructure, reducing the need for third-party LLM integrations. ([splunk.com](https://www.splunk.com/en_us/blog/conf-splunklive/supercharge-insights-with-generative-ai-splunk-ai-assistant-for-spl-is-now-ga.html?utm_source=openai))\n- The Splunk MCP and hosted models are positioned to support “Agentic Ops” by enabling direct agent access to Splunk search and discovery tools with minimal integration effort. ([splunk.com](https://www.splunk.com/en_us/blog/platform/splunk-platform-building-the-data-foundation-for-agentic-ai.html?utm_source=openai))\n- Splunk also announced the Splunk AI Assistant for SPL is GA, using hosted LLMs to generate SPL from natural language and to explain SPL queries, available to Splunk Cloud Platform customers globally. ([splunk.com](https://www.splunk.com/en_us/blog/conf-splunklive/supercharge-insights-with-generative-ai-splunk-ai-assistant-for-spl-is-now-ga.html?utm_source=openai))\n- The AI Assistant for SPL is hosted by Splunk, with data inputs/outputs owned by the customer and assurance that data is not sent to external third-party LLM services. ([splunk.com](https://www.splunk.com/en_us/blog/conf-splunklive/supercharge-insights-with-generative-ai-splunk-ai-assistant-for-spl-is-now-ga.html?utm_source=openai))\n- This hosted-model rollout follows prior AI enhancements announced at .conf24 (2024), which expanded AI capabilities across Observability Cloud and Security, including SPL-assisted workflows. ([splunk.com](https://www.splunk.com/en_us/newsroom/press-releases/2024/conf24-splunk-introduces-advanced-ai-enhancements-for-observability-security-and-it-service-intelligence.html?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '345_2026-02-10', 'news_date': '2026-02-10', 'output': '- Alphabet sold $20 billion in a seven-tranche U.S. dollar bond offering to fund AI infrastructure, with the sale reported around February 10, 2026 and settlement set for February 13, 2026. ([streetinsider.com](https://www.streetinsider.com/Corporate%2BNews/Alphabet%2Bprices%2B%2420%2Bbillion%2Bmulti-tranche%2Bbond%2Boffering/25970761.html?utm_source=openai))\n\n- The USD offering comprises seven tranches with maturities ranging from 2029 to 2066, totaling $20 billion. Tranche sizes include $2.5B (2029), $3B (2031), $3B (2033), $4.25B (2036), $1.5B (2046), $4B (2056), and $1.75B (2066). ([streetinsider.com](https://www.streetinsider.com/Corporate%2BNews/Alphabet%2Bprices%2B%2420%2Bbillion%2Bmulti-tranche%2Bbond%2Boffering/25970761.html?utm_source=openai))\n\n- Coupon rates and yields across the tranches span from 3.700% to 5.750%; yields to maturity run from about 3.829% (2029) to 5.794% (2066), with spreads to Treasuries of roughly 27 to 95 basis points. ([streetinsider.com](https://www.streetinsider.com/Corporate%2BNews/Alphabet%2Bprices%2B%2420%2Bbillion%2Bmulti-tranche%2Bbond%2Boffering/25970761.html?utm_source=openai))\n\n- Net proceeds to Alphabet after underwriting discounts and before expenses are approximately $19.85 billion. ([streetinsider.com](https://www.streetinsider.com/Corporate%2BNews/Alphabet%2Bprices%2B%2420%2Bbillion%2Bmulti-tranche%2Bbond%2Boffering/25970761.html?utm_source=openai))\n\n- Underwriters for the USD issue are BofA Securities, Goldman Sachs & Co. LLC, and J.P. Morgan Securities LLC, serving as joint global coordinators and joint book-running managers. ([streetinsider.com](https://www.streetinsider.com/Corporate%2BNews/Alphabet%2Bprices%2B%2420%2Bbillion%2Bmulti-tranche%2Bbond%2Boffering/25970761.html?utm_source=openai))\n\n- The notes carry Moody’s Aa2 rating and S&P AA+ rating (both with stable outlooks). ([streetinsider.com](https://www.streetinsider.com/Corporate%2BNews/Alphabet%2Bprices%2B%2420%2Bbillion%2Bmulti-tranche%2Bbond%2Boffering/25970761.html?utm_source=openai))\n\n- Use of proceeds is to fund Alphabet’s accelerated AI spending, including data centers and AI chips. ([economictimes.indiatimes.com](https://economictimes.indiatimes.com/markets/us-stocks/news/google-parent-alphabet-sells-bonds-worth-20-billion-to-fund-ai-spending/articleshow/128157139.cms))\n\n- Alphabet is guiding toward up to about $185 billion in capital expenditures for 2026 to support AI infrastructure, a context that helped spur the bond sale. ([economictimes.indiatimes.com](https://economictimes.indiatimes.com/markets/us-stocks/news/google-parent-alphabet-sells-bonds-worth-20-billion-to-fund-ai-spending/articleshow/128157139.cms))\n\n- In addition to the USD sale, Alphabet marketed European-denominated bonds in Sterling (about £5.5B) and Swiss francs (CHF 3.1B), including a rare 100-year Sterling issue; the European deals followed the USD sale. ([bloomberg.com](https://www.bloomberg.com/news/articles/2026-02-10/alphabet-begins-selling-multi-tranche-debut-swiss-franc-bond-mlgb76p8?utm_source=openai))\n\n- The February 2026 activity fits a broader pattern of AI-debt fundraising among large tech companies, with Oracle having raised about $25 billion in notes earlier in February 2026. ([economictimes.indiatimes.com](https://economictimes.indiatimes.com/markets/us-stocks/news/google-parent-alphabet-sells-bonds-worth-20-billion-to-fund-ai-spending/articleshow/128157139.cms?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '346_2026-02-10', 'news_date': '2026-02-10', 'output': '- Alibaba’s DAMO Academy unveiled RynnBrain, an embodied AI foundation model designed to act as the “brain” for robots, released as a family of open-source models. ([alibaba-damo-academy.github.io](https://alibaba-damo-academy.github.io/RynnBrain.github.io/?utm_source=openai))\n\n- RynnBrain is built on Alibaba’s Qwen3-VL vision-language foundation, grounding reasoning in physical space and time. ([alibaba-damo-academy.github.io](https://alibaba-damo-academy.github.io/RynnBrain.github.io/?utm_source=openai))\n\n- The open-source family includes multiple sizes and configurations: 2B, 8B, and a 30B Mixture-of-Experts (MoE) variant, with specialized forms such as Plan, Nav, and CoP. ([huggingface.co](https://huggingface.co/Alibaba-DAMO-Academy/RynnBrain-2B?utm_source=openai))\n\n- Weights and inference code for the RynnBrain family were released in early February 2026, with hosting on platforms like Hugging Face and GitHub. ([huggingface.co](https://huggingface.co/Alibaba-DAMO-Academy/RynnBrain-2B?utm_source=openai))\n\n- RynnBrain delivers egocentric perception, diverse spatiotemporal localization, and physics-grounded reasoning to enable real-world task planning for robots. ([huggingface.co](https://huggingface.co/Alibaba-DAMO-Academy/RynnBrain-Nav-8B?utm_source=openai))\n\n- It incorporates spatiotemporal memory, allowing robots to recall prior states and timing of tasks to resume or reorder actions, a capability highlighted in its demonstrations. ([en.tmtpost.com](https://en.tmtpost.com/news/7873991?utm_source=openai))\n\n- Alibaba notes RynnBrain’s training and efficiency innovations, including data scales (reported training data exceeding 20 million pairs) and a custom training acceleration approach called RynnScale. ([en.tmtpost.com](https://en.tmtpost.com/news/7873991?utm_source=openai))\n\n- In benchmark testing, RynnBrain reportedly achieves state-of-the-art results on 16 embodied-cognition benchmarks, outperforming Google’s Gemini Robotics ER 1.5 and Nvidia’s Cosmos-Reason2 in these tests. ([caixinglobal.com](https://www.caixinglobal.com/2026-02-11/alibaba-unveils-robot-brain-raising-stakes-in-embodied-ai-race-102413424.html?utm_source=openai))\n\n- The launch has drawn coverage framing it as part of China’s broader push into embodied AI and robotics, a focus in competition with Western players. ([bloomberg.com](https://www.bloomberg.com/news/newsletters/2025-09-25/alibaba-is-turning-into-china-s-hottest-ai-play?utm_source=openai))\n\n- Alibaba opened the full RynnBrain line (and seven other models) as open-source, signaling an intent to accelerate global robotics research and adoption through community access. ([en.tmtpost.com](https://en.tmtpost.com/news/7873991?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '347_2026-02-10', 'news_date': '2026-02-10', 'output': '- Cisco expands AgenticOps innovations across its portfolio, with agentic capabilities set to start rolling out in February 2026. ([newsroom.cisco.com](https://newsroom.cisco.com/content/r/newsroom/en/us/a/y2026/m02/cisco-expands-agenticops-innovations-across-portfolio.html?utm_source=openai))\n\n- Rollout scope includes Campus, Branch, and Industrial networking capabilities beginning February 2026; Data Center capabilities with controlled availability in June 2026; and Service Provider capabilities in beta. ([newsroom.cisco.com](https://newsroom.cisco.com/content/r/newsroom/en/us/a/y2026/m02/cisco-expands-agenticops-innovations-across-portfolio.html?utm_source=openai))\n\n- In Campus/Branch/Industrial, new capabilities include Autonomous Troubleshooting, Continuous Optimization, Trusted Validation, Experience Metrics, and Agentic Workflow Creation. ([newsroom.cisco.com](https://newsroom.cisco.com/content/r/newsroom/en/us/a/y2026/m02/cisco-expands-agenticops-innovations-across-portfolio.html?utm_source=openai))\n\n- Cisco notes the AgenticOps platform ingests live signals from across Cisco Networking, Security Cloud Control, Nexus One, Splunk, and more (including ThousandEyes and Secure Firewall) to enable context-aware, agentic execution. ([newsroom.cisco.com](https://newsroom.cisco.com/content/r/newsroom/en/us/a/y2026/m02/cisco-expands-agenticops-innovations-across-portfolio.html?utm_source=openai))\n\n- Observability integration includes AI Agent Monitoring in Splunk Observability Cloud to visualize agent workflows, with integration to Cisco AI Defense; generally available February 25. ([investor.cisco.com](https://investor.cisco.com/news/news-details/2026/Cisco-Expands-AgenticOps-Innovations-Across-Portfolio/default.aspx?utm_source=openai))\n\n- A new security pillar—AI Defense expansion—aims to govern agentic AI and provide runtime protections for agent tooling, complemented by AI-aware SASE to optimize AI-driven traffic and protection. ([newsroom.cisco.com](https://newsroom.cisco.com/content/r/newsroom/en/us/a/y2026/m02/cisco-redefines-security-for-the-agentic-era.html?utm_source=openai))\n\n- Security capabilities include AI BOM (bill of materials), MCP catalog, and multi-turn red teaming, plus runtime guardrails to reduce risk of compromise or manipulation in agentic workflows; post-quantum cryptography enhancements are highlighted for IOS XE 26. ([newsroom.cisco.com](https://newsroom.cisco.com/content/r/newsroom/en/us/a/y2026/m02/cisco-redefines-security-for-the-agentic-era.html?utm_source=openai))\n\n- General availability for selected security and agentic features is targeted for May 2026. ([investor.cisco.com](https://investor.cisco.com/news/news-details/2026/Cisco-Redefines-Security-for-the-Agentic-Era-with-AI-Defense-Expansion-and-AI-Aware-SASE/default.aspx?utm_source=openai))\n\n- Cisco emphasizes cross-domain telemetry (Networking, Security Cloud Control, Nexus One, Splunk) and unified visibility to support reliable, scalable agentic operations across cloud, on-premises, air-gapped industrial environments, data centers, and service provider networks. ([investor.cisco.com](https://investor.cisco.com/news/news-details/2026/Cisco-Expands-AgenticOps-Innovations-Across-Portfolio/default.aspx?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '348_2026-02-10', 'news_date': '2026-02-10', 'output': '- Bretton AI (formerly Greenlite AI) announced a $75 million Series B and a company rebrand to Bretton AI. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- The round was led by Sapphire Ventures, with new investment from TIAA Ventures and continued support from Greylock, Y Combinator, Thomson Reuters Ventures, and Canvas Ventures. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- The funding is intended to expand Bretton AI’s AI offerings across financial crime domains, deepen regulatory engagement, and accelerate adoption across more geographies and compliance categories. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Bretton AI provides AI agents that perform AML/KYC/KYB investigations, sanctions screening, and ongoing monitoring, operating across existing compliance tools with a proprietary Trust Infrastructure for governance and risk management. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- The platform is designed to reduce reliance on traditional staffing by pairing AI agents with human analysts to handle high-volume, regulated tasks more quickly and with audit-ready outputs. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- The rebrand to Bretton AI is positioned as building an AI standard for financial crime, with the name chosen as an homage to Bretton Woods and its implications for global financial stability. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- The company highlights notable customer results tied to its platform, including a $15B+ institution cutting BPO spend by $5.35M in Year 1, a Fortune 500 firm halving onboarding time, and a FDIC-regulated bank reducing loan origination time by 90%. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Founder and CEO Will Lawrence leads Bretton AI; prior experience includes work on anti-money laundering at Facebook and compliance at Paxos. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Bretton AI is headquartered in Wilmington, Delaware (address listed as 251 Little Falls Drive). ([cbinsights.com](https://www.cbinsights.com/company/greenlite?utm_source=openai))\n- The company emphasizes regulatory collaboration and existing relationships with OCC, FDIC, and Federal Reserve-regulated banks, alongside partnerships with fintech platforms such as Mercury, Gusto, and Upgrade. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '349_2026-02-10', 'news_date': '2026-02-10', 'output': '- Thomson Reuters announced it has acquired Noetica, Inc., an AI-native platform for corporate transaction intelligence, with the deal disclosed on February 10, 2026; the plan is to integrate Noetica’s analytics across CoCounsel AI. ([thomsonreuters.com](https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence))\n\n- Noetica is a New York–based startup founded in 2022 that transforms transaction-deal data into structured market intelligence for deal professionals. ([thomsonreuters.com](https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence))\n\n- Noetica’s platform provides secure, single-tenant deployments of transaction deal data and offers vertical AI specialization, qualitative/quantitative benchmarking, natural-language term search, term-trend analysis, and deal-level risk signals. ([thomsonreuters.com](https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence))\n\n- Thomson Reuters plans to integrate Noetica’s AI-native analytics and structured market intelligence across CoCounsel, enabling deal professionals to benchmark key deal terms during negotiations, assist with legal drafting, and gain deeper transaction data analysis throughout the deal workflow. ([thomsonreuters.com](https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence))\n\n- Leading Thomson Reuters executive Raghu Ramanathan and Noetica CEO Dan Wertman publicly framed the acquisition as a transformative step to bring real-time market insights and market-aware AI to legal work. ([thomsonreuters.com](https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence))\n\n- Noetica’s AI capabilities are evidenced by a “deep bench” of AI talent—senior AI scientists, PhDs, and seasoned legal/finance experts—added to Thomson Reuters’ vertical AI capabilities. ([thomsonreuters.com](https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence))\n\n- Thomson Reuters Ventures was an early investor in Noetica’s Series A, underscoring a prior strategic alignment between the two companies. ([thomsonreuters.com](https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence))\n\n- Noetica maintains a large-scale data graph, with claims of 1.5 billion terms and real-time processing across dozens of transaction types, bolstering its benchmarked transaction data capabilities. ([thomsonreuters.com](https://www.thomsonreuters.com/en/press-releases/2026/february/thomson-reuters-acquires-noetica-inc-the-ai-native-platform-for-corporate-transaction-intelligence))', 'model_provider': 'openai'},
{'event_id': '350_2026-02-10', 'news_date': '2026-02-10', 'output': "- Runway raises $315 million in a Series E at a $5.3 billion valuation to scale its AI video capabilities and pre-train the next generation of world models. ([techcrunch.com](https://techcrunch.com/2026/02/10/ai-video-startup-runway-raises-315m-at-5-3b-valuation-eyes-more-capable-world-models/))\n\n- The round was led by General Atlantic; investors include Nvidia, Fidelity Management & Research, AllianceBernstein, and Mirae Asset. [Additional backers noted include Adobe Ventures, Emphatic Capital, Felicis, Premji, and AMD Ventures.] ([techcrunch.com](https://techcrunch.com/2026/02/10/ai-video-startup-runway-raises-315m-at-5-3b-valuation-eyes-more-capable-world-models/))\n\n- Gen 4.5, Runway’s video model, enables high-definition video generation from prompts and adds native audio, long-form and multi-shot generation, character consistency, and advanced editing tools. ([techcrunch.com](https://techcrunch.com/2026/02/10/ai-video-startup-runway-raises-315m-at-5-3b-valuation-eyes-more-capable-world-models/))\n\n- Runway states Gen 4.5 outperformed Google's and OpenAI’s video-generation offerings on the Video Arena leaderboard. ([techcrunch.com](https://techcrunch.com/2025/12/11/runway-releases-its-first-world-model-adds-native-audio-to-latest-video-model/))\n\n- Runway released its first world model, GWM-1, on December 11, 2025, with three variants: GWM-Worlds, GWM-Robotics, and GWM-Avatars; it’s built on Gen-4.5 and uses frame-by-frame prediction to simulate reality in real time. ([techcrunch.com](https://techcrunch.com/2025/12/11/runway-releases-its-first-world-model-adds-native-audio-to-latest-video-model/))\n\n- GWM-1 is described as more general than competitors like Genie-3 and is aimed at training agents for domains such as robotics and life sciences. ([techcrunch.com](https://techcrunch.com/2025/12/11/runway-releases-its-first-world-model-adds-native-audio-to-latest-video-model/))\n\n- GWM-Worlds runs at 24 frames per second at 720p and provides an interactive, physics-aware environment. ([techcrunch.com](https://techcrunch.com/2025/12/11/runway-releases-its-first-world-model-adds-native-audio-to-latest-video-model/))\n\n- Runway signed a deal with CoreWeave to expand compute capacity for its AI video workloads. ([techcrunch.com](https://techcrunch.com/2026/02/10/ai-video-startup-runway-raises-315m-at-5-3b-valuation-eyes-more-capable-world-models/))\n\n- The company plans to grow its team to roughly 140 people across research, engineering, and go-to-market roles. ([techcrunch.com](https://techcrunch.com/2026/02/10/ai-video-startup-runway-raises-315m-at-5-3b-valuation-eyes-more-capable-world-models/))\n\n- Adobe Ventures participated in the funding, and Runway has an Adobe partnership in its ecosystem. ([techcrunch.com](https://techcrunch.com/2026/02/10/ai-video-startup-runway-raises-315m-at-5-3b-valuation-eyes-more-capable-world-models/))", 'model_provider': 'openai'},
{'event_id': '351_2026-02-10', 'news_date': '2026-02-10', 'output': '- Qureos closes a $5 million seed round to scale its AI-driven hiring platform Iris across global markets, led by Prosus Ventures and Salica Oryx Fund. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed?utm_source=openai))\n- Iris automates the end-to-end recruitment workflow (sourcing, screening, interviewing) with AI-enriched candidate profiles and multi-channel distribution across more than 2,000 job boards. ([aipressroom.com](https://aipressroom.com/qureos-5m-hiring-platform/?utm_source=openai))\n- Candidate screening is performed in under 15 seconds, and AI-led interviews are delivered via audio or video formats tailored to each role. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed?utm_source=openai))\n- Qureos is UAE-based and built in the Middle East, operating across MENA and globally, including the United States. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed?utm_source=openai))\n- The platform has reportedly crossed 1,000 enterprise and public-sector customers, including Qatar Airways and Dubai Economy and Tourism. ([aipressroom.com](https://aipressroom.com/qureos-5m-hiring-platform/?utm_source=openai))\n- Funds will be used to deepen AI capabilities, expand the go-to-market team, and accelerate geographic expansion through enterprise and agency partnerships. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed?utm_source=openai))\n- Lead investors on the round are Prosus Ventures and Salica Oryx Fund; other participants include Oraseya Capital, PlusVC, F6 Ventures, BDev Ventures, Sunny Side Venture Partners, and Daniel Tyre, with follow-on investments from COTU Ventures and Globivest. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed?utm_source=openai))\n- Iris’ architecture supports integration with ATS/HRIS systems (e.g., SAP SuccessFactors, Oracle) or can operate as a standalone solution. ([wasssl.com](https://wasssl.com/qureos-raises-5-million-seed-round-to-cut-hiring-time-and-redefine-recruitment-speed/?utm_source=openai))\n- Market context notes localization and nationalization requirements across GCC markets, which Qureos has positioned its AI to navigate as it scales. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed?utm_source=openai))\n- Some coverage reports the seed as $5M with total funding mentioned in some outlets around $8M, but the primary disclosed figure is $5M; exact total across rounds is variably reported. ([aipressroom.com](https://aipressroom.com/qureos-5m-hiring-platform/?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '352_2026-02-10', 'news_date': '2026-02-10', 'output': '- Gather AI raises $40 million in Series B led by Smith Point Capital Management to scale its Physical AI Platform for Global Logistics. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))\n- The round includes participation from Bain Capital Ventures, Tribeca Venture Partners, Bling Capital, Dundee Venture Capital, XRC Ventures, and new investor The Hillman Company; total funding to date is about $74 million. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))\n- The funding follows a year in which Gather AI doubled its operational footprint and bookings grew 250%. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))\n- The platform digitizes pallets, tasks, and movements in real time using AI-powered vision on consumer-grade hardware such as drones and material handling equipment, with zero infrastructure changes required. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))\n- Gather AI customers achieve 99.9% inventory accuracy, up to 80% reduction in manual counting, and up to 5x productivity; ROI is typically under six months. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))\n- Named customers include GEODIS, NFI Industries, Kwik Trip, Axon, dnata, Barrett Distribution, and Langham Logistics. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))\n- Gather AI describes its offering as the world’s first Physical Intelligence Platform—the system of record for intralogistics operations across warehouses, plants, and yards. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))\n- The investment will fund expansion to hundreds of additional facilities globally and the development of predictive capabilities for proactive inventory management; engineering and customer success teams will scale accordingly. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))\n- The company plans to showcase its platform at Manifest and MODEX in 2026 as part of its growth push. ([gather.ai](https://www.gather.ai/news/gather-ai-raises-40m-led-by-smith-point-capital-management-to-scale-its-physical-ai-platform-for-global-logistics-2))', 'model_provider': 'openai'},
{'event_id': '353_2026-02-10', 'news_date': '2026-02-10', 'output': '- Simple AI announced a seed funding round of $14 million for its voice AI agent platform, led by First Harmonic with participation from Y Combinator, Massive Tech Ventures, and True Ventures, to build selling-focused voice agents for consumer brands. (Sources: Business Wire, VentureBeat) ([businesswire.com](https://www.businesswire.com/news/home/20260210526354/en/))\n\n- The funding will fund development of Simple AI’s voice agent platform, including custom generative AI models and analytics/customer insights offerings for businesses of all sizes. (Sources: Business Wire, VentureBeat) ([businesswire.com](https://www.businesswire.com/news/home/20260210526354/en/))\n\n- The platform ingests a customer’s full product catalog (SKUs, metadata, pricing) and uses real-time customer data on each call to personalize interactions and even place orders, with transcripts, summaries, and actionable insights produced per call. (Sources: Business Wire, VentureBeat) ([businesswire.com](https://www.businesswire.com/news/home/20260210526354/en/))\n\n- Latency is designed to stay under 850 milliseconds across the entire call pipeline (voice activity detection, model inference, transcription, and text-to-speech) to enable natural conversations. (Sources: Business Wire, VentureBeat) ([businesswire.com](https://www.businesswire.com/news/home/20260210526354/en/))\n\n- Simple AI’s voice agents are positioned to automate inbound and outbound B2C calls for sales and support, with use cases spanning multiple product categories (e.g., steak, self storage, home insurance). (Sources: VentureBeat, Business Wire) ([venturebeat.com](https://venturebeat.com/business/simple-ai-announces-14m-from-first-harmonic-and-y-combinator-to-build-voice-ai-agents-that-sell))\n\n- The seed round highlights the participation of Y Combinator, signaling early-stage backing alongside public accelerator ties; First Harmonic is the lead investor. (Sources: Business Wire, VentureBeat) ([businesswire.com](https://www.businesswire.com/news/home/20260210526354/en/))\n\n- On performance, Simple AI claims its agents convert and upsell up to 30% more often than trained live reps. (Sources: VentureBeat, Business Wire) ([venturebeat.com](https://venturebeat.com/business/simple-ai-announces-14m-from-first-harmonic-and-y-combinator-to-build-voice-ai-agents-that-sell))\n\n- Founders Catheryn Li and Zach Kamran met and began building the technology in the YC ecosystem, and they lead Simple AI as co-founders/ceo & co-founder/CTO respectively. (Sources: VentureBeat, Business Wire) ([venturebeat.com](https://venturebeat.com/business/simple-ai-announces-14m-from-first-harmonic-and-y-combinator-to-build-voice-ai-agents-that-sell))\n\n- First Harmonic’s JJ Fliegelman characterized the founders as quietly building a complete, in-house voice AI stack, illustrating strong technical execution behind the seed. (Sources: VentureBeat, Business Wire) ([venturebeat.com](https://venturebeat.com/business/simple-ai-announces-14m-from-first-harmonic-and-y-combinator-to-build-voice-ai-agents-that-sell))', 'model_provider': 'openai'},
{'event_id': '354_2026-02-10', 'news_date': '2026-02-10', 'output': '- Core development: IBM announces a next-generation FlashSystem portfolio—FlashSystem 5600, 7600, and 9600—introducing autonomous storage powered by agentic AI (FlashSystem.ai); general availability is planned for March 6, 2026. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- Model scale and density: FlashSystem 5600 targets compact, edge-friendly deployments with up to 2.5 PBe of effective capacity in a 1U form factor and up to 2.6M IOPs; FlashSystem 7600 targets higher performance in 2U with up to 7.2 PBe and up to 4.3M IOPs; FlashSystem 9600 targets mission-critical scales in 2U with up to 11.8 PBe and up to 6.3M IOPs. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- Storage footprint and efficiency: The new FlashSystem lineup claims up to a 30%–75% reduction in storage footprint versus the previous generation, depending on model. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- FlashSystem.ai overview: FlashSystem.ai delivers AI-powered data services with self-service operations, enabling autonomous, agentic AI-driven management and optimization of storage resources. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- AI-driven automation scope: The AI model behind FlashSystem.ai is trained on tens of billions of data points from telemetry and real-world data, enabling thousands of automated decisions per day without human intervention. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- Proactive workload management: The latest generation supports proactive tuning and intelligent placement of workloads, as well as non-disruptive data mobility across storage devices, including third-party arrays. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- FlashCore Module (FCM5): All new FlashSystem models use the fifth-generation FlashCore Module, delivering up to 105 TB per drive and hardware-accelerated per‑I/O analytics for enhanced security and performance. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- Ransomware and threat protection: The FCM5 enables hardware-assisted ransomware detection and autonomous recovery actions, with AI-driven threat detection claimed to occur rapidly (e.g., under 60 seconds in demonstrations). ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- Compliance/audit efficiency: IBM asserts that FlashSystem.ai capabilities can reduce audit and compliance documentation time by half via AI-generated, explainable operational reasoning. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))\n\n- Analyst perspective and market context: IDC and other analysts are cited as acknowledging that advanced AI capabilities in FlashSystem can help meet SLAs and improve resilience for mission-critical workloads. ([prnewswire.com](https://www.prnewswire.com/news-releases/ibm-introduces-autonomous-storage-with-new-flashsystem-portfolio-powered-by-agentic-ai-302683212.html))', 'model_provider': 'openai'},
{'event_id': '355_2026-02-10', 'news_date': '2026-02-10', 'output': '- Core development: Vibrant Capital launches as an operator-led investment and company-building platform to scale AI in real economy workflows, introducing Growth Partners Network, Vibrant Studio incubator, and the nonprofit CIO Fellows Society. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- Leadership: Shadman Zafar named Chief Executive Officer; previously held executive roles at Citigroup, JPMorgan Chase, Barclays, and Verizon; named as an inventor on 100+ patents/applications. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- Market focus: Will pursue workflow-embedded AI across financial services and insurance, healthcare and life sciences, industrial and manufacturing, transportation and logistics, and business and IT services; initial operations in the United States, Europe, and the Gulf Cooperation Council. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- Growth Partners Network: A partnership platform for AI-native companies already delivering enterprise value; seasoned CIO operators select real-outcome ventures and help scale adoption with repeatable deployment playbooks and operator-led go-to-market. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- Couchbase partnership: Couchbase, led by CEO BJ Schaknowski, joins Vibrant Capital’s ecosystem to support AI workloads with a unified platform across operational, analytical, mobile, and AI use cases. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- Skan.ai partnership: Skan.ai, led by CEO and co-founder Avinash Misra, contributes to blueprinting where AI creates value via an enterprise Context Graph enabling real-time guidance and scalable execution. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- Vibrant Studio: An incubator for founders who aim to ship into real operations, with Vibrant acting as an active co-builder to drive enterprise readiness, early traction, and scaling beyond pilots. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- HaiIntel: Vibrant Capital’s first Studio launch, introducing an AI-driven Outcome-as-a-Service model that embeds intelligence at execution and unifies AI platforms, production-grade systems, and execution guidance to deliver scale. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- CIO Fellows Society: A nonprofit initiative that convenes global technology executives to mentor future CIOs and promote practical patterns for scaling AI responsibly in complex environments. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))\n- Launch timing and place: Public announcement dated February 10, 2026, with operations identified in Dallas and Fort Worth, Texas. ([prnewswire.com](https://www.prnewswire.com/news-releases/vibrant-capital-launches-to-scale-ai-in-the-real-economy-302682153.html))', 'model_provider': 'openai'},
{'event_id': '356_2026-02-10', 'news_date': '2026-02-10', 'output': '- Altruist launches AI-enabled tax planning features within its Hazel platform for advisors. ([wealthmanagement.com](https://www.wealthmanagement.com/artificial-intelligence/altruist-launches-ai-powered-tax-planning-feature-in-hazel-platform-for-advisors?utm_source=openai))\n- The new capability uses client documents to generate personalized tax strategies and interactive scenario modeling for planning conversations. ([wealthmanagement.com](https://www.wealthmanagement.com/artificial-intelligence/altruist-launches-ai-powered-tax-planning-feature-in-hazel-platform-for-advisors?utm_source=openai))\n- Hazel’s AI tax planning reads 1040s, pay stubs, account statements, emails, meeting notes, and custodial/CRM data to output client-ready tax recommendations in minutes. ([investmentnews.com](https://www.investmentnews.com/goria/custodian/altruist-expands-hazel-ai-with-tax-planning-capabilities/265211?utm_source=openai))\n- The feature is available to advisory firms regardless of custodian, marking Hazel as a cross-custodian planning aid. ([investmentnews.com](https://www.investmentnews.com/goria/custodian/altruist-expands-hazel-ai-with-tax-planning-capabilities/265211?utm_source=openai))\n- Hazel AI tax planning is priced at $60 per seat per month when sold as a standalone product. ([wealthmanagement.com](https://www.wealthmanagement.com/artificial-intelligence/altruist-launches-ai-powered-tax-planning-feature-in-hazel-platform-for-advisors?utm_source=openai))\n- The announcement follows Hazel’s broader platform trajectory, including a 2025 custodial integration that lets Hazel access real-time custodial data for personalized advice. ([altruist.com](https://altruist.com/news/altruist-debuts-industry-first-custodial-integration/?utm_source=openai))\n- The February 10, 2026 launch date places this feature in the context of a notable market moment when Reuters reported AI disruption fears had created a buying opportunity in US software stocks. ([uk.finance.yahoo.com](https://uk.finance.yahoo.com/news/ai-disruption-fears-create-buying-174109629.html?utm_source=openai))\n- On the same day, shares of major wealth-management firms fell as investors repriced risk to AI-enabled disruption, with Schwab, Raymond James, and LPL Financial among the notable movers. ([investing.com](https://www.investing.com/news/stock-market-news/us-brokerages-fall-as-aidriven-rout-extends-to-financials-4497583?utm_source=openai))\n- Altruist’s Hazel platform is described as automating core advisory tasks and expanding planning capabilities, with tax planning as the first expanded planning module in Hazel. ([wealthmanagement.com](https://www.wealthmanagement.com/artificial-intelligence/altruist-launches-ai-powered-tax-planning-feature-in-hazel-platform-for-advisors?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '357_2026-02-10', 'news_date': '2026-02-10', 'output': '- WHAT IS NEW: Israel-based FINQ launches two U.S. ETFs that are fully managed by artificial intelligence, a first-of-its-kind in the SEC-approved AI-managed ETF space. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- FUNDS AND TICKERS: FINQ FIRST U.S. Large Cap AI-Managed Equity ETF (AIUP) and FINQ DOLLAR NEUTRAL U.S. Large Cap AI-Managed Equity ETF (AINT). ([finqai.com](https://www.finqai.com/etfs?utm_source=openai))\n\n- HOW THE AI WORKS: The funds rely on FINQ’s proprietary adaptive AI framework that ranks all 500 stocks in the S&P 500 daily to guide portfolio construction, weighting, and rebalancing. ([finqai.com](https://www.finqai.com/etfs?utm_source=openai))\n\n- AI AS SOLE MANAGER: Portfolio decisions are fully selected and managed by FINQ’s AI model, with human involvement limited to oversight and governance. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- DATA INPUTS: The AI framework uses market, financial, and textual data to generate daily stock rankings that drive holdings. ([finqai.com](https://www.finqai.com/etfs?utm_source=openai))\n\n- FUND STRUCTURE: The ETFs are sub-advised by FINQ AI LLC and advised by Tidal Investments LLC; FINQ filed a preliminary prospectus for both funds on August 27, 2025. ([finanznachrichten.de](https://www.finanznachrichten.de/nachrichten-2025-08/66286469-finq-files-preliminary-prospectus-for-two-ai-managed-u-s-large-cap-etfs-200.htm?utm_source=openai))\n\n- LAUNCH DATE/LOCALE: The announcement and launch occurred February 10, 2026, with reporting from Tel Aviv and Providence, Rhode Island. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- INDUSTRY CONTEXT: AI-managed ETFs are a nascent sub-sector; prior attempts (e.g., Simplify Asset Management in 2023) and caution from analysts about AI stock-selection turnover have been noted. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- AINT SPECIFIC: AINT is described as the dollar-neutral U.S. Large Cap AI-Managed Equity ETF, complementing AIUP’s long/tilt strategy. ([finanznachrichten.de](https://www.finanznachrichten.de/nachrichten-2025-08/66286469-finq-files-preliminary-prospectus-for-two-ai-managed-u-s-large-cap-etfs-200.htm?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '358_2026-02-10', 'news_date': '2026-02-10', 'output': '- What is new: Bretton AI, formerly Greenlite AI, announces a $75 million Series B and a rebrand to Bretton AI. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Round details: Series B led by Sapphire Ventures; participants include Greylock Partners, Y Combinator, Thomson Reuters Ventures, Canvas Ventures; new investor TIAA Ventures; stated that valuation was not disclosed. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Brand change: The company is renaming from Greenlite AI to Bretton AI to reflect its expanded focus on AI for financial crime operations. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Product/use-case: AI agents deployed to automate financial crime workflows (KYC, KYB, AML, sanctions investigations, ongoing monitoring) and designed to operate inside customers’ existing compliance tools. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Customers and deployment: Claims adoption across OCC-regulated banks and other FDIC-regulated banks in the United States; customers mentioned include Mercury, Gusto, and Upgrade. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Trust Infrastructure: Agents built on a proprietary Trust Infrastructure with model risk management, continuous AI evaluations, and rigorous QA to be audit-ready and regulator-aligned. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- ROI and customer outcomes: Reported results include a $15B+ institution cutting BPO spend by $5.35M in Year 1; a Fortune 500 company reduced onboarding time by 50%; an FDIC-regulated bank reduced loan origination timelines by up to 90%. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Market positioning: The company frames financial crime as the breakout AI use case for financial services. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))\n- Announcement date and branding rationale: Announcement dated February 10, 2026; the branding references Bretton Woods to signal an industry-standard approach for trusted AI in regulated finance. ([bretton.com](https://www.bretton.com/blog/bretton-raises-75m-series-b-to-modernize-aml-kyc-staffing?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '359_2026-02-10', 'news_date': '2026-02-10', 'output': '- Core development: FINQ announces two U.S. ETFs that will be solely managed by AI—AIUP (FINQ FIRST U.S. LARGE CAP AI-MANAGED EQUITY ETF) and AINT (FINQ DOLLAR NEUTRAL U.S. LARGE CAP AI-MANAGED EQUITY ETF)—described as the first of their kind to be approved in the U.S. by the SEC. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- Management model: Portfolio decisions for both funds will be fully selected and managed by FINQ’s AI model, with human involvement limited to oversight and governance. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- SEC approval: The funds were approved by the U.S. Securities and Exchange Commission, marking the first approval of AI-only ETFs in this category. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- Fund focus: The ETFs target U.S. large-cap equity exposure. ([sec.gov](https://www.sec.gov/Archives/edgar/data/1742912/000199937125010597/finq-485apos_080525.htm?utm_source=openai))\n\n- Daily ranking mechanism: FINQ’s AI framework continually ranks all 500 stocks in the S&P 500 to inform portfolio holdings, weights, and rebalancing decisions. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- Data inputs: The AI uses a combination of market, financial, and textual data to drive stock rankings and portfolio construction. ([pymnts.com](https://www.pymnts.com/artificial-intelligence-2/2026/fund-manager-finq-lets-ai-run-us-etfs/?utm_source=openai))\n\n- Founding perspective: FINQ’s CEO Eldad Tamir characterizes the system as “data-only” and claims it can process vast data without human biases or emotions. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- Industry context: Analysts cautioned that AI-driven stock selection has had a mixed track record, with some prior funds experiencing high turnover and closures. ([investing.com](https://www.investing.com/news/stock-market-news/israels-finq-launches-us-etfs-managed-solely-by-ai-4497608?utm_source=openai))\n\n- Corporate structure and partnerships: FINQ is Israel-based and publicly positions the firm as a sub-adviser to these ETFs, with distribution handled by a U.S. partner; the company’s site describes its structure (FINQ Digital, Ltd.) and adviser/distributor relationships. ([finqai.com](https://finqai.com/?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '361_2026-02-10', 'news_date': '2026-02-10', 'output': '- Multiverse Computing is in talks to raise about €500 million in fresh funding, valuing the company at over €1.5 billion, with closing expected in the first half of 2026. ([cincodias.elpais.com](https://cincodias.elpais.com/companias/2026-02-10/multiverse-negocia-una-ronda-de-500-millones-para-su-ia-cuantica-y-consolidarse-como-unicornio.html?utm_source=openai))\n- The new round’s investors have not been disclosed, and terms are not yet finalized. ([cincodias.elpais.com](https://cincodias.elpais.com/companias/2026-02-10/multiverse-negocia-una-ronda-de-500-millones-para-su-ia-cuantica-y-consolidarse-como-unicornio.html?utm_source=openai))\n- Multiverse Computing is a Spanish AI software company known for quantum-inspired AI model compression, notably its CompactifAI technology. ([techcrunch.com](https://techcrunch.com/2025/06/12/multiverse-computing-raises-215m-for-tech-that-could-radically-slim-ai-costs/?utm_source=openai))\n- CompactifAI compresses large language models by up to 95% with only 2–3% precision loss, enabling faster inference and potential edge deployments. ([techcrunch.com](https://techcrunch.com/2025/06/12/multiverse-computing-raises-215m-for-tech-that-could-radically-slim-ai-costs/?utm_source=openai))\n- The company is headquartered in Donostia-San Sebastián, Spain, with a global customer base and a focus on scaling its compression technology. ([techcrunch.com](https://techcrunch.com/2025/06/12/multiverse-computing-raises-215m-for-tech-that-could-radically-slim-ai-costs/?utm_source=openai))\n- The Bloomberg-originated reporting (cited by Cinco Días) places the funding discussions as of early 2026, with a close planned for H1 2026. ([cincodias.elpais.com](https://cincodias.elpais.com/companias/2026-02-10/multiverse-negocia-una-ronda-de-500-millones-para-su-ia-cuantica-y-consolidarse-como-unicornio.html?utm_source=openai))\n- Context: the reported talks reflect continued European investor appetite for AI and deep-tech startups, including potential unicorns, amid a broader funding surge in Europe. ([ft.com](https://www.ft.com/content/523ee7a9-489a-4f68-8aae-3f98c336b479?utm_source=openai))\n- No use-of-proceeds or strategic allocation details have been disclosed publicly as part of the talks. ([cincodias.elpais.com](https://cincodias.elpais.com/companias/2026-02-10/multiverse-negocia-una-ronda-de-500-millones-para-su-ia-cuantica-y-consolidarse-como-unicornio.html?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '362_2026-02-10', 'news_date': '2026-02-10', 'output': '- Multiverse Computing is in discussions to raise about €500 million in fresh funding, valuing the company at more than €1.5 billion, with closing expected in the first half of 2026. ([bloomberg.com](https://www.bloomberg.com/news/articles/2026-02-10/ai-firm-multiverse-said-to-hit-1-5-billion-value-with-new-funds))\n- The company reported annual recurring revenue of about €100 million as of January 2026. ([bloomberg.com](https://www.bloomberg.com/news/articles/2026-02-10/ai-firm-multiverse-said-to-hit-1-5-billion-value-with-new-funds))\n- The round is expected to bring in new backers while existing investors (HP Tech Ventures, Forgepoint Capital, CDP Venture Capital) and the Spanish SETT regulator-backed backer may participate. ([bloomberg.com](https://www.bloomberg.com/news/articles/2026-02-10/ai-firm-multiverse-said-to-hit-1-5-billion-value-with-new-funds))\n- If completed, the financing would position Multiverse Computing as a European AI unicorn (valuation above €1 billion). ([bloomberg.com](https://www.bloomberg.com/news/articles/2026-02-10/ai-firm-multiverse-said-to-hit-1-5-billion-value-with-new-funds))\n- Core technology: CompactifAI, a quantum-inspired AI model compression platform that turns weight matrices into tensor networks to shrink LLMs while aiming to preserve performance. ([tradingview.com](https://www.tradingview.com/news/reuters.com%2C2025%3Anewsml_L4N3SE1A5%3A0-spain-s-multiverse-raises-217-million-for-compressing-ai-models/?utm_source=openai))\n- CompactifAI targets compression of open-source LLMs, including models in the Llama family (e.g., Llama 4 Scout, Llama 3.3 70B, Llama 3.1 8B) and Mistral Small 3.1, with DeepSeek R1 planned soon. ([techcrunch.com](https://techcrunch.com/2025/06/12/multiverse-computing-raises-215m-for-tech-that-could-radically-slim-ai-costs/?utm_source=openai))\n- The founders/leadership include CTO Román Orús and CEO Enrique Lizaso Olmos. ([techcrunch.com](https://techcrunch.com/2025/06/12/multiverse-computing-raises-215m-for-tech-that-could-radically-slim-ai-costs/?utm_source=openai))\n- In open demonstrations, compressed models run significantly faster and cheaper: 4–12x faster inference and 50–80% cost reductions, with potential edge-device deployment. ([techcrunch.com](https://techcrunch.com/2025/06/12/multiverse-computing-raises-215m-for-tech-that-could-radically-slim-ai-costs/?utm_source=openai))\n- The company has publicly highlighted a portfolio of more than 100 customers and a broad patent footprint (e.g., around 160 patents). ([techcrunch.com](https://techcrunch.com/2025/06/12/multiverse-computing-raises-215m-for-tech-that-could-radically-slim-ai-costs/?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '364_2026-02-10', 'news_date': '2026-02-10', 'output': '- NIST announces Phase II SBIR awards totaling $3.19 million to eight U.S. small businesses across seven states to advance AI, biotechnology, semiconductors, quantum and related technologies; awards are for 24 months of R&D prototyping, with Phase III following upon completion. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))\n\n- AMAG Consulting LLC (Schenectady, NY) — $400,000 — Scanning electron microscope software (SimuSEM) will be updated to model charging effects and magnetic interactions, enabling more accurate imaging via added magnetic field and surface roughness modeling. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))\n\n- Applied Imaging Solutions LLC (Quincy, MA) — $400,000 — Develop SWIR-HSI/AI imaging to measure quality attributes in NISTmAb-producing NISTCHO cell cultures, enabling contactless monitoring of cell viability and metabolites for biopharmaceutical manufacturing. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))\n\n- Calimetrix LLC (Madison, WI) — $399,998 — Create phantoms for multimodality MRI/CT measurements that mimic steatotic tissue (fatty liver) to improve cross-platform imaging accuracy. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))\n\n- HighRI Optics Inc. (Oakland, CA) — $399,858.96 — Develop a binary pseudo-random array for enhanced optical imaging, including a calibration standard and data reconstruction software to boost resolution. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))\n\n- Icarus Quantum Inc. (Boulder, CO) — $400,000 — Create a turnkey photon source based on semiconductor quantum dots for scalable, on-chip, high-efficiency photon sources to accelerate quantum interconnect development. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))\n\n- MyExposome Inc. (Philadelphia, PA) — $395,814.74 — Use silicone wristbands as personal PFAS exposure monitors with solvent-free extraction methods to expand detectable PFAS in research and consumer markets. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))\n\n- ObjectSecurity LLC (San Diego, CA) — $399,908.58 — Develop Operational Technology AI — NIST Compliance Tool (OTAI-NCT) to evaluate cybersecurity practices of hardware/software makers and generate a cyber-hygiene score using public datasets. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))\n\n- Universal Schedule and Booking LLC (Harpers Ferry, WV) — $400,000 — Phase II project to optimize home energy-load profiles without extra sensors, accelerating adoption of energy-tracking tech for a more efficient grid. ([nist.gov](https://www.nist.gov/news-events/news/2026/02/nist-awards-over-3-million-small-businesses-advancing-ai-biotechnology))', 'model_provider': 'openai'},
{'event_id': '365_2026-02-10', 'news_date': '2026-02-10', 'output': '- Core development: Berget AI closes a funding round totaling SEK 24 million to build a sovereign AI infrastructure platform in Europe, led by Luminar Ventures with participation from Wellstreet and Norrsken Evolve (reported as SEK 24M; €2.1M; USD 2.4M). ([wellstreet.se](https://wellstreet.se/story/wellstreet-invests-in-berget-ai))\n- Date of announcement: Funding news published February 2, 2026. ([wellstreet.se](https://wellstreet.se/story/wellstreet-invests-in-berget-ai))\n- Lead investors and participants: Round led by Luminar Ventures, with participation from Wellstreet and Norrsken Evolve. ([tech.eu](https://tech.eu/2026/02/02/swedish-startup-berget-ai-lands-2-1m-as-demand-grows-for-sovereign-ai-in-europe//))\n- Strategic objective: Funds to accelerate development and commercialization of Berget AI’s sovereign AI platform, offering a European alternative to global cloud providers. ([wellstreet.se](https://wellstreet.se/story/wellstreet-invests-in-berget-ai))\n- Platform description: Full-stack AI platform enabling development of applications using open language models on sovereign infrastructure located in Sweden, ensuring data stays under European jurisdiction. ([wellstreet.se](https://wellstreet.se/story/wellstreet-invests-in-berget-ai))\n- Founders: Christian Landgren and Andreas Lundmark spearhead Berget AI. ([wellstreet.se](https://wellstreet.se/story/wellstreet-invests-in-berget-ai))\n- Product timeline: Berget AI’s first service was launched shortly before summer 2025, with ongoing development and market interest. ([wellstreet.se](https://wellstreet.se/story/wellstreet-invests-in-berget-ai))\n- Regulatory and market context: The funding aligns with Europe’s emphasis on data sovereignty amid the EU AI Act and broader sovereign-cloud initiatives. ([wellstreet.se](https://wellstreet.se/story/wellstreet-invests-in-berget-ai))\n- Positioning and value proposition: Berget AI aims to be a turnkey alternative to US hyperscalers for developers needing data sovereignty, open-model flexibility, and operational control. ([tech.eu](https://tech.eu/2026/02/02/swedish-startup-berget-ai-lands-2-1m-as-demand-grows-for-sovereign-ai-in-europe//))\n- Industry backdrop: European sovereign cloud and sovereignty-focused AI infrastructure are growing priorities as policymakers and investors push for local AI data control. ([techradar.com](https://www.techradar.com/pro/europe-is-going-all-in-on-sovereign-cloud-investment-to-triple-by-2027?utm_source=openai))', 'model_provider': 'openai'},
{'event_id': '366_2026-02-10', 'news_date': '2026-02-10', 'output': '- Anytime AI announced the launch of Anytime AI 2.0, featuring “Talk to Teddy” and Agentic AI workflows for plaintiff-law firms (Feb 10, 2026, White Plains, NY). ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/10/3235214/0/en/Anytime-AI-Launches-2-0-Introducing-Talk-to-Teddy-and-Agentic-Workflows-for-Law-Firms.html?utm_source=openai))\n\n- “Talk to Teddy” is a unified chat interface that reasons through complex case files and retrieves real-time open-web insights, all within a fully encrypted, closed environment. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/10/3235214/0/en/Anytime-AI-Launches-2-0-Introducing-Talk-to-Teddy-and-Agentic-Workflows-for-Law-Firms.html?utm_source=openai))\n\n- Anytime AI 2.0 provides enterprise-grade security and privacy, including HIPAA compliance and 256-bit encryption; client data remains private to the firm and is not used to train public models. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/10/3235214/0/en/Anytime-AI-Launches-2-0-Introducing-Talk-to-Teddy-and-Agentic-Workflows-for-Law-Firms.html?utm_source=openai))\n\n- The platform introduces Agentic Workflows with AI Agents that autonomously orchestrate multi-step tasks for high-stakes litigation, such as medical chronologies, demand letters, discovery responses, and legal research/drafting. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/10/3235214/0/en/Anytime-AI-Launches-2-0-Introducing-Talk-to-Teddy-and-Agentic-Workflows-for-Law-Firms.html?utm_source=openai))\n\n- Target practice areas for 2.0 are Personal Injury, Nursing Home Negligence, and Medical Malpractice. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/10/3235214/0/en/Anytime-AI-Launches-2-0-Introducing-Talk-to-Teddy-and-Agentic-Workflows-for-Law-Firms.html?utm_source=openai))\n\n- The release emphasizes eliminating tool-switching friction by delivering a single, high-performance hub that combines internal document analysis with external legal research and avoids unencrypted third-party tools. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/10/3235214/0/en/Anytime-AI-Launches-2-0-Introducing-Talk-to-Teddy-and-Agentic-Workflows-for-Law-Firms.html?utm_source=openai))\n\n- CEO Teddy Wu (Lingfei Wu) stated that the goal of 2.0 was to remove the “clicks” and focus on delivering results in high-stakes litigation. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/10/3235214/0/en/Anytime-AI-Launches-2-0-Introducing-Talk-to-Teddy-and-Agentic-Workflows-for-Law-Firms.html?utm_source=openai))\n\n- Company context: Anytime AI positions itself as an all-in-one legal AI platform built for plaintiff lawyers, with ongoing 2.0-related blog updates on its site. ([anytimeai.ai](https://www.anytimeai.ai/?utm_source=openai))\n\n- Context on product lineage: 2.0 follows prior generations, including the 4th generation launched Oct 30, 2024 and the 5th generation launched Apr 16, 2025, showing progressive feature expansion in the platform’s evolution. ([prnewswire.com](https://www.prnewswire.com/news-releases/anytime-ai-unveils-fourth-generation-of-its-ai-legal-assistant-introducing-four-industry-first-features-302291099.html?utm_source=openai))', 'model_provider': 'openai'}]